In [ ]:
!pip install wandb
!wandb login

wandb: Currently logged in as: jd202 (use `wandb login --relogin` to force relogin)


In [ ]:
# Download and extract the image dataset

!rm -r data
!gdown --id 13MRa51YRElF8n88Kh7RMglJCqSrTmrcg

import zipfile
with zipfile.ZipFile(f'./data.zip', 'r') as zip:
    zip.extractall('.')
!rm data.zip

Downloading...
From: https://drive.google.com/uc?id=13MRa51YRElF8n88Kh7RMglJCqSrTmrcg
To: /content/data.zip
100% 15.1M/15.1M [00:00<00:00, 53.6MB/s]


In [ ]:
# Establishing constants

DATA_DIRECTORY = './data'
BATCH_SIZE=128
LEARNING_RATE=0.0025
EPOCHS=150
COLOR_MODE='grayscale' # Can be 'rgb' or 'grayscale', amongst other optionse
IMAGE_SIZE=(96, 96) # Width x height, can be larger than input image 
INPUT_SHAPE=IMAGE_SIZE + (1 if COLOR_MODE == "grayscale" else 3,) # Width x height x colour profile (RGB = 3 channels, grayscale = 1 channel)

# RANDOM_CROP_WIDTH = 96
# RANDOM_CROP_HEIGHT = 96

ROTATION_RANGE = 15
HORIZONTAL_FLIP = True
WIDTH_SHIFT_RANGE=0
HEIGHT_SHIFT_RANGE=0

# assert RANDOM_CROP_WIDTH <= IMAGE_SIZE[0]
# assert RANDOM_CROP_HEIGHT <= IMAGE_SIZE[1]

In [ ]:
import glob
from PIL import Image
import pandas as pd
import numpy as np
import tensorflow as tf

files = glob.glob(DATA_DIRECTORY + '/**/*.jpg')
files = sorted(files)

data_df = pd.read_csv(f'{DATA_DIRECTORY}/metadata_lookup.csv')

print(files[3863])
print(data_df.loc[3863])

# image_data = np.array([np.array(Image.open(fname)) for fname in files])

# label_data_age, label_data_gender = data_df[['age']].to_numpy(), data_df[['gender']].to_numpy()

./data/1/1281_age_30_gender_1.jpg
file_name    1/1281_age_30_gender_1.jpg
age                                  30
gender                                1
Name: 3863, dtype: object


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf


data_df = pd.read_csv(f'{DATA_DIRECTORY}/metadata_lookup.csv')
data_df = data_df.sample(frac=1).reset_index(drop=True)

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0, 
    horizontal_flip=HORIZONTAL_FLIP, 
    validation_split=0.25,
    rotation_range=ROTATION_RANGE,
    width_shift_range=WIDTH_SHIFT_RANGE,
    height_shift_range=HEIGHT_SHIFT_RANGE
    )

val_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0, 
    # horizontal_flip=HORIZONTAL_FLIP, 
    validation_split=0.25,
    # rotation_range=ROTATION_RANGE
    )

data_generator_from_df = data_generator.flow_from_dataframe(
    dataframe=data_df, 
    directory=DATA_DIRECTORY, 
    x_col='file_name', 
    y_col=['age', 'gender'], 
    # has_ext=True, #Deprecated
    batch_size=BATCH_SIZE, 
    shuffle=True,
    target_size=IMAGE_SIZE, 
    color_mode=COLOR_MODE, 
    class_mode='raw',
    subset='training', 
    seed=50)

val_data_generator_from_df = val_data_generator.flow_from_dataframe(
    dataframe=data_df, 
    directory=DATA_DIRECTORY, 
    x_col='file_name', 
    y_col=['age', 'gender'], 
    # has_ext=True, #Deprecated
    batch_size=BATCH_SIZE, 
    shuffle=True,
    target_size=IMAGE_SIZE, 
    color_mode=COLOR_MODE, 
    class_mode='raw',
    subset='validation', 
    seed=50)


def custom_generator(generator):
  while True:
    k = generator.next()
    yield(k[0], {'DenseAge': k[1][:, 0], 'DenseGender': k[1][:, 1]})

Found 3750 validated image filenames.
Found 1250 validated image filenames.


In [ ]:
# import numpy as np

# image_data, label_data = [], []
# batch_index = 0

# while batch_index <= data_generator_from_df.batch_index:
#     data = data_generator_from_df.next()
#     image_data.extend(data[0])
#     label_data.extend(data[1])
#     batch_index = batch_index + 1
# image_data, label_data = np.array(image_data), np.array(label_data)

In [ ]:
# print(image_data.shape)
# print(label_data.shape)
# print(label_data[0])

# label_data_age, label_data_gender = label_data[:, 0], label_data[:, 1]

# print(label_data_age)
# print(label_data_gender)

In [ ]:
c = custom_generator(val_data_generator_from_df)

# print(next(c)[1])

d = next(c)
print(d)
# for i in range(1):
#   print(next(c))

# p = 0
# while True:
#   i = next(c)
#   # print(i.shape)
#   p += 1
#   print(p)

(array([[[[0.6       ],
         [0.5921569 ],
         [0.54901963],
         ...,
         [0.4784314 ],
         [0.5137255 ],
         [0.57254905]],

        [[0.6039216 ],
         [0.61960787],
         [0.59607846],
         ...,
         [0.49803925],
         [0.5254902 ],
         [0.5686275 ]],

        [[0.58431375],
         [0.6156863 ],
         [0.59607846],
         ...,
         [0.50980395],
         [0.5411765 ],
         [0.5921569 ]],

        ...,

        [[0.11764707],
         [0.1254902 ],
         [0.10588236],
         ...,
         [0.2627451 ],
         [0.2627451 ],
         [0.2627451 ]],

        [[0.1254902 ],
         [0.1254902 ],
         [0.1137255 ],
         ...,
         [0.25882354],
         [0.25882354],
         [0.25490198]],

        [[0.13725491],
         [0.11764707],
         [0.10980393],
         ...,
         [0.26666668],
         [0.2627451 ],
         [0.24705884]]],


       [[[0.09411766],
         [0.09803922],
         [0.1

In [ ]:
#Check GPU exists and is in use

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17221974790461172795
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11320098816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4184633890588405526
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
xla_global_id: 416903419
]


In [ ]:
# Define the model using the Functional API

from tensorflow.keras import layers
import keras

# tf.debugging.set_log_device_placement(True) # Uncomment to verify Keras is running the model on the GPU

input = keras.Input(shape=INPUT_SHAPE)
# x = layers.RandomCrop(RANDOM_CROP_HEIGHT, RANDOM_CROP_WIDTH) # Yes, the parameters do go height then width, for some reason... :|

# def resblock(x, kernelsize, filters):
#     fx = layers.Conv2D(filters=filters, kernel_size=kernelsize, activation='relu', padding='same')(x)
#     fx = layers.BatchNormalization()(fx)
#     fx = layers.Conv2D(filters=filters, kernel_size=kernelsize, padding='same')(fx)
#     out = layers.Add()([x,fx])
#     out = layers.ReLU()(out)
#     # out = layers.Dropout(0.5)(out)
#     out = layers.BatchNormalization()(out)
#     return out


x = layers.Conv2D(32, (3, 3))(input)
x = layers.MaxPooling2D()(x)

x = layers.Conv2D(48, (3, 3))(x)
x = layers.MaxPooling2D()(x)

x = layers.Conv2D(64, (3, 3))(x)
x = layers.MaxPooling2D()(x)


x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dense(1024, activation="relu")(x)
# x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)

out_age = layers.Dense(1, activation="relu", name="DenseAge")(x) # Using ReLU activation to skip negative values, as the age will never be negative
out_gender = layers.Dense(1, activation="sigmoid", name="DenseGender")(x) # Using sigmoid to limit 0 <= x <= 1
model = keras.models.Model(inputs = input, outputs = [out_age, out_gender])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 96, 96, 1)]  0           []                               
                                                                                                  
 conv2d_3 (Conv2D)              (None, 94, 94, 32)   320         ['input_2[0][0]']                
                                                                                                  
 max_pooling2d_3 (MaxPooling2D)  (None, 47, 47, 32)  0           ['conv2d_3[0][0]']               
                                                                                                  
 conv2d_4 (Conv2D)              (None, 45, 45, 32)   9248        ['max_pooling2d_3[0][0]']        
                                                                                            

In [ ]:
import wandb
from wandb.keras import WandbCallback

model.compile(
    loss = {
        "DenseAge": 'mean_absolute_error',
        # "DenseGender": tf.keras.losses.BinaryCrossentropy(from_logits=False)
        "DenseGender": 'binary_crossentropy'
    },

    metrics = {
        "DenseAge": 'mean_absolute_error',
        "DenseGender": 'binary_accuracy'
    },

    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
)

wandb_config = {
    'learning_rate': LEARNING_RATE,
    'epochs': EPOCHS,
    'batch_size': BATCH_SIZE,
    'color_mode': COLOR_MODE,
    'image_size': IMAGE_SIZE,
    # 'random_crop_width': RANDOM_CROP_WIDTH,
    # 'random_crop_height': RANDOM_CROP_HEIGHT,
    'rotation_range': ROTATION_RANGE,
    # 'rescaling_factor': RESCALING_FACTOR,
    'horizontal_flip': HORIZONTAL_FLIP,
    'width_shift_range': WIDTH_SHIFT_RANGE,
    'height_shift_range': HEIGHT_SHIFT_RANGE
    # 'vertical_flip': VERTICAL_FLIP,
    # 'zoom_range': ZOOM_RANGE,
    # 'width_shift_range': WIDTH_SHIFT_RANGE,
    # 'height_shift_range': HEIGHT_SHIFT_RANGE,
    
}

wandb.init(project="ml2-group-k", entity="ml2-group-k", config=wandb_config)

STEP_SIZE_TRAIN=data_generator_from_df.n//data_generator_from_df.batch_size
STEP_SIZE_VALID=val_data_generator_from_df.n//val_data_generator_from_df.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

history = model.fit(
    x = custom_generator(data_generator_from_df),
    steps_per_epoch = STEP_SIZE_TRAIN,
    validation_steps = STEP_SIZE_VALID,
    validation_data=custom_generator(val_data_generator_from_df),
    epochs = EPOCHS,
    callbacks=[WandbCallback()])


# history = model.fit_generator(
#     generator=custom_generator(data_generator_from_df),
#     steps_per_epoch = STEP_SIZE_TRAIN,
#     # validation_steps = 50,
#     # validation_split = 0.25,
#     validation_data=custom_generator(val_data_generator_from_df),
#     validation_steps=STEP_SIZE_VALID,
#     epochs = EPOCHS,
#     callbacks=[WandbCallback()])

# data_generator_from_df

DenseAge_loss,██▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
DenseAge_mean_absolute_error,██▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
DenseGender_binary_accuracy,▁▁▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇██▇▇███████████
DenseGender_loss,█▄▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_DenseAge_loss,█▇▄▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_DenseAge_mean_absolute_error,█▇▄▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_DenseGender_binary_accuracy,▁▁▅▆▆▆▇▇▇▇▇▇█████▇█▇▇███████▇███▇███████
val_DenseGender_loss,██▆▄▄▄▃▂▃▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁▁▂▁▂▁▂▂▂▂▁▂▂▂▁▁
val_loss,█▇▄▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


Epoch 1/150
29/29 [==============================] - 8s 256ms/step - loss: 23.0123 - DenseAge_loss: 21.6608 - DenseGender_loss: 1.3516 - DenseAge_mean_absolute_error: 21.6608 - DenseGender_binary_accuracy: 0.5022 - val_loss: 17.1008 - val_DenseAge_loss: 16.4006 - val_DenseGender_loss: 0.7002 - val_DenseAge_mean_absolute_error: 16.4006 - val_DenseGender_binary_accuracy: 0.4947 - _timestamp: 1647825766.0000 - _runtime: 13.0000
Epoch 2/150
29/29 [==============================] - 7s 245ms/step - loss: 17.2027 - DenseAge_loss: 16.3025 - DenseGender_loss: 0.9003 - DenseAge_mean_absolute_error: 16.3025 - DenseGender_binary_accuracy: 0.5178 - val_loss: 14.8806 - val_DenseAge_loss: 14.1466 - val_DenseGender_loss: 0.7340 - val_DenseAge_mean_absolute_error: 14.1466 - val_DenseGender_binary_accuracy: 0.4902 - _timestamp: 1647825773.0000 - _runtime: 20.0000
Epoch 3/150
29/29 [==============================] - 7s 241ms/step - loss: 16.3883 - DenseAge_loss: 15.6062 - DenseGender_loss: 0.7821 - Dense

In [ ]:
print('Final val MAE on age:', str(history.history['val_DenseAge_mean_absolute_error'][-1]))
print('Final val binary accuracy on gender:', str(history.history['val_DenseGender_binary_accuracy'][-1]))

Final val MAE on age: 7.204899787902832
Final val binary accuracy on gender: 0.8600713014602661


In [ ]:
# import datetime
# import json
# from pathlib import Path

# final_loss = str(round(history.history['val_loss'][-1], 5))
# final_datetime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
# file_name = f'{final_datetime} - {final_loss}'

# Path(f'/outputs/{file_name}').mkdir(parents=True, exist_ok=True)

# model.save(f'./outputs/{file_name}/model.h5')

# with open(f'./outputs/{file_name}/config.json', 'w', encoding='utf8') as j:
#     j.write(json.dumps(model.get_config()))
    
# with open(f'./outputs/{file_name}/results.json', 'w', encoding='utf8') as j:
#     j.write(json.dumps(history.history))

In [ ]:
# Garbage Collector - use it like gc.collect()
import gc

# Custom Callback To Include in Callbacks List At Training Time
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [ ]:
# !zip -r outputs.zip ./outputs